# Component Demonstration Notebook

A notebook to demonstrate some of the features of the Component class

Author: gavin.treseder@essentialenergy.com.au

In [1]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np 
import pandas as pd

import plotly.express as px

from pof.component import Component

In [2]:
comp = Component().set_demo()

In [3]:
comp.mc_timeline(t_end=200)

100%|██████████| 100/100 [00:03<00:00, 27.04it/s]


In [4]:
cdf = comp.expected_cdf(t_end=1000)

In [6]:
erc = comp.expected_risk_cost()
erc

{'fast_aging': {'inspection': {'time': array([  0,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,
           65,  70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125,
          130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190,
          195, 200]),
   'cost': array([ 0., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
          50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
          50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
          50., 50.]),
   'cumulative_cost': array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,
           450.,  500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,
           900.,  950., 1000., 1050., 1100., 1150., 1200., 1250., 1300.,
          1350., 1400., 1450., 1500., 1550., 1600., 1650., 1700., 1750.,
          1800., 1850., 1900., 1950., 2000.])},
  'on_condition_repair': {'time': array([  0,  25,  27,  29,  30,  31,  32,  33,  35,  36,  37,  38,  4

In [100]:
time = list(np.linspace(0,200, 201))

n_time = 201

failure_modes = []
tasks = []
times = []
costs = []
cumulative_costs = []

n_fm = len(erc)

for fm, fm_values in erc.items():

    for task, task_values in fm_values.items():
        failure_mode = np.full(n_time, fm)
        task = np.full(n_time, task)
        cost = np.full(n_time, 0)
        cost[task_values['time']] = task_values['cost']
        cumulative_cost = cost.cumsum()

        failure_modes = np.append(failure_modes, failure_mode)
        tasks = np.append(tasks, task)
        costs = np.append(costs, cost)
        times = np.append(times, time)
        cumulative_costs = np.append(cumulative_costs, cumulative_cost)


df = pd.DataFrame(
    dict(
        failure_mode=failure_modes,
        task=tasks,
        time=times,
        cost=costs,
        cumulative_cost=cumulative_costs,
    )
)

In [107]:
px.area(
    df.reset_index(),
    x="time",
    y="cumulative_cost",
    color="task",
    line_group = 'failure_mode'
)

In [6]:
profile = comp.fm['random'].expected_cost()
profile['risk'] = comp.fm['random'].expected_risk()

In [7]:
%timeit comp.fm['slow_aging'].expected_cost_df()

140 ms ± 30.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit comp.fm['slow_aging'].expected_risk_cost_df()

27.7 ms ± 9.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
rc = comp.fm['slow_aging'].expected_risk_cost()

In [18]:
new_index = pd.Index(np.arange(0, 200, 1), name="time")

t = rc['inspection']['time']
c = rc['inspection']['cost']

In [28]:
tc = dict(task=[], time=[], cost=[])

for k, v in rc.items():
    tc['task'] = np.append(tc['task'], np.full(len(v['time']), k))
    for m in ['time', 'cost']:
        time = tc[m]
        
        

        tc[m] = np.append(tc[m], v[m])

array([ 0. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 49.5])

In [9]:
df = comp.fm['random'].expected_risk_cost_df()
df.index.name = 'task'
df = df.reset_index()

In [10]:
df = comp.fm['slow_aging'].expected_risk_cost_df()

In [12]:

px.line(
    df.reset_index(),
    x="time",
    y="cumulative_cost",
    color="task",
    line_group =
)

In [63]:
df

,time,cost,cumulative_cost,1
inspection,5,50,NaN,NaN
inspection,10,50,NaN,NaN
inspection,15,50,NaN,NaN
inspection,20,50,NaN,NaN
inspection,25,50,NaN,NaN
...,...,...,...,...
on_condition_repair,200,1,NaN,NaN
cm,135,50,NaN,NaN
cm,180,50,NaN,NaN
risk,132,50000,NaN,NaN


In [13]:
comp.fm['random'].expected_risk_cost_df()

,time,cost,cumulative_cost
task,,,
inspection,0,0,0
inspection,5,50,50
inspection,10,50,100
inspection,15,50,150
inspection,20,50,200
...,...,...,...
on_condition_repair,200,2,37
cm,0,0,0
cm,60,50,50


In [22]:
%timeit profile = comp.fm['random'].expected_risk()

277 µs ± 96.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%timeit profile = comp.fm['random'].expected_cost()

566 ms ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit profile = comp.fm['random'].tasks['inspection'].expected_costs()

336 ms ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
task = comp.fm['random'].tasks['inspection']

In [29]:

%timeit {t:task.t_completion.count(t) * task.cost / 1 for t in task.t_completion}

311 ms ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
pd.DataFrame(
    dict(
        Task='adsf',
        Time=time,
        Cost=cost
    )
)

,Task,Time,Cost
0,adsf,5,100
1,adsf,10,100
2,adsf,15,100
3,adsf,20,100
4,adsf,25,100
5,adsf,30,100
6,adsf,35,100
7,adsf,40,100
8,adsf,45,100
9,adsf,50,100


In [35]:
{t:task.t_completion.count(t) * task.cost / 1 for t in task.t_completion}

{5: 5000.0,
 10: 5000.0,
 15: 5000.0,
 20: 5000.0,
 25: 5000.0,
 30: 5000.0,
 35: 5000.0,
 40: 5000.0,
 45: 5000.0,
 50: 5000.0,
 55: 5000.0,
 60: 5000.0,
 65: 5000.0,
 70: 5000.0,
 75: 5000.0,
 80: 5000.0,
 85: 5000.0,
 90: 5000.0,
 95: 5000.0,
 100: 5000.0,
 105: 5000.0,
 110: 5000.0,
 115: 5000.0,
 120: 5000.0,
 125: 5000.0,
 130: 5000.0,
 135: 5000.0,
 140: 5000.0,
 145: 5000.0,
 150: 5000.0,
 155: 5000.0,
 160: 5000.0,
 165: 5000.0,
 170: 5000.0,
 175: 5000.0,
 180: 5000.0,
 185: 5000.0,
 190: 5000.0,
 195: 5000.0,
 200: 5000.0}

{'inspection': {5: 50.0,
  10: 50.0,
  15: 50.0,
  20: 50.0,
  25: 50.0,
  30: 50.0,
  35: 50.0,
  40: 50.0,
  45: 50.0,
  50: 50.0,
  55: 50.0,
  60: 50.0,
  65: 50.0,
  70: 50.0,
  75: 50.0,
  80: 50.0,
  85: 50.0,
  90: 50.0,
  95: 50.0,
  100: 50.0,
  105: 50.0,
  110: 50.0,
  115: 50.0,
  120: 50.0,
  125: 50.0,
  130: 50.0,
  135: 50.0,
  140: 50.0,
  145: 50.0,
  150: 50.0,
  155: 50.0,
  160: 50.0,
  165: 50.0,
  170: 50.0,
  175: 50.0,
  180: 50.0,
  185: 50.0,
  190: 50.0,
  195: 50.0,
  200: 50.0},
 'on_condition_repair': {180: 3.0,
  95: 1.0,
  50: 3.0,
  60: 1.0,
  145: 1.0,
  195: 1.0,
  185: 1.0,
  160: 1.0,
  90: 2.0,
  170: 1.0,
  5: 2.0,
  125: 1.0,
  35: 1.0,
  30: 2.0,
  110: 2.0,
  15: 2.0,
  190: 1.0,
  65: 1.0,
  80: 2.0,
  55: 2.0,
  155: 1.0,
  10: 1.0,
  70: 1.0,
  120: 1.0,
  20: 1.0,
  45: 1.0},
 'cm': {},
 'risk': {}}